# Problem statement
Depending on the data of a customer, deciding wheather to give loan to the customer or not. 
1. NPA: non performimg asset: eg. bamk gave loan to a customer but the customer is unable to pay it. Or it is a loan account when DPD>90 days.
if DPD=0 then NDA (non deliquent account) (non default account)
if DPD is in (0 to 30) SMA1 (Standard monitoring account)
if (3 1to 60): SMA2 (Standard monitoring account)
if (60 to 90): SMA3 (Standard monitoring account)
if >90  t0 180 : NPA
DPD>180: written-off (which is not present at all)

NPA improve= Loan portfolio quality of the bnk improve ==market sentiment will iprove == stock price will improve

*******************
1. Disbursed Amout: loan amount given to the customer
2. OSP: Outstanding Principle(the balance about to be paid to the bank by customer is OSP), and OSP should be zero at the end.
3. DPD: days paased due (extra days which are passed after the due date), DPd  shouln't be greater than zero.
4. PAR: Prtfolio at risk: when OSP,DPD is grteater than zero.





# NPA
GNPA: Gross NPA (3-5 %)= OSP default
NNPA: net NPA= (0.01 to 0.06%)= Provisioning amount subtracted

Bank qualuty assess: GNPA value

In [1]:
import os 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.metrics import r2_score,accuracy_score,confusion_matrix,classification_report,precision_recall_fscore_support
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
import time 

In [2]:
data_link1="https://github.com/ravina029/datasets/raw/main/case_study1.xlsx"
data_link2="https://github.com/ravina029/datasets/raw/main/case_study2.xlsx"

unseen_data="https://github.com/ravina029/datasets/raw/main/Unseen_Dataset.xlsx"


In [3]:
d1=pd.read_excel(data_link1)
d2=pd.read_excel(data_link2)

In [4]:
df1=d1.copy()
df2=d2.copy()

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [6]:
df1.isnull().sum().sum()

0

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   time_since_recent_payment     51336 non-null  int64  
 2   time_since_first_deliquency   51336 non-null  int64  
 3   time_since_recent_deliquency  51336 non-null  int64  
 4   num_times_delinquent          51336 non-null  int64  
 5   max_delinquency_level         51336 non-null  int64  
 6   max_recent_level_of_deliq     51336 non-null  int64  
 7   num_deliq_6mts                51336 non-null  int64  
 8   num_deliq_12mts               51336 non-null  int64  
 9   num_deliq_6_12mts             51336 non-null  int64  
 10  max_deliq_6mts                51336 non-null  int64  
 11  max_deliq_12mts               51336 non-null  int64  
 12  num_times_30p_dpd             51336 non-null  int64  
 13  n

In [8]:
df2.isnull().sum().sum()

0

In [9]:
df1.sample(5)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
7853,7854,4,2,2,0,0,0.0,0.0,0.5,0.5,...,0,1,0,0,1,2,2,2,56,19
48673,48674,2,0,2,1,0,0.5,0.0,1.0,0.0,...,0,1,0,0,0,0,2,1,7,5
10898,10899,4,0,4,2,0,0.5,0.0,1.0,0.0,...,0,3,1,0,0,1,3,0,10,5
46426,46427,2,1,1,0,0,0.0,0.0,0.5,0.5,...,0,0,0,2,0,2,0,0,29,28
6474,6475,1,0,1,0,0,0.0,0.0,1.0,0.0,...,0,0,1,0,0,1,0,0,7,7


In [10]:
df2.sample(5)

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
127,128,43,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,-99999.000,1,0,others,others,688,P2
44279,44280,118,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.667,0.0,-99999.000,0,0,PL,PL,650,P4
14589,14590,106,-99999,-99999,0,-99999,0,0,0,0,...,1.0,0.000,1.0,-99999.000,0,0,CC,HL,680,P2
37824,37825,97,-99999,-99999,0,-99999,0,0,0,0,...,0.0,1.000,0.0,1.411,0,0,PL,others,686,P2
40542,40543,48,17,10,7,56,26,0,2,2,...,0.0,0.000,0.0,10.458,1,0,CC,CC,687,P2


In [11]:
df1.shape

(51336, 26)

In [12]:
df1[df1['Age_Oldest_TL'] ==-99999].shape[0]


40

In [13]:
# -99999(data integrety issue) is null value, we have 40 nulll values 

# remove these null values 
df1=df1.loc[df1['Age_Oldest_TL'] !=-99999]


In [14]:
df1.shape

(51296, 26)

In [15]:
null_val={}
for col in df2.columns:
    null_val[col]=df2[df2[col] ==-99999].shape[0]


print(null_val)
null_val_more_10000={col:null_val[col] for col in null_val if null_val[col]>10000}
print(null_val_more_10000)
print("number of columns to be removed:",len(null_val_more_10000))

{'PROSPECTID': 0, 'time_since_recent_payment': 4291, 'time_since_first_deliquency': 35949, 'time_since_recent_deliquency': 35949, 'num_times_delinquent': 0, 'max_delinquency_level': 35949, 'max_recent_level_of_deliq': 0, 'num_deliq_6mts': 0, 'num_deliq_12mts': 0, 'num_deliq_6_12mts': 0, 'max_deliq_6mts': 12890, 'max_deliq_12mts': 10832, 'num_times_30p_dpd': 0, 'num_times_60p_dpd': 0, 'num_std': 0, 'num_std_6mts': 0, 'num_std_12mts': 0, 'num_sub': 0, 'num_sub_6mts': 0, 'num_sub_12mts': 0, 'num_dbt': 0, 'num_dbt_6mts': 0, 'num_dbt_12mts': 0, 'num_lss': 0, 'num_lss_6mts': 0, 'num_lss_12mts': 0, 'recent_level_of_deliq': 0, 'tot_enq': 6321, 'CC_enq': 6321, 'CC_enq_L6m': 6321, 'CC_enq_L12m': 6321, 'PL_enq': 6321, 'PL_enq_L6m': 6321, 'PL_enq_L12m': 6321, 'time_since_recent_enq': 6321, 'enq_L12m': 6321, 'enq_L6m': 6321, 'enq_L3m': 6321, 'MARITALSTATUS': 0, 'EDUCATION': 0, 'AGE': 0, 'GENDER': 0, 'NETMONTHLYINCOME': 0, 'Time_With_Curr_Empr': 0, 'pct_of_active_TLs_ever': 0, 'pct_opened_TLs_L6m_of

1. we have very high number of missing values and removing these can lead to information loss 
2. we will use some techniwue to impute these nulll values if null values are less or remove the column with very high number of values.

In [16]:
df2=df2.drop(columns=null_val_more_10000.keys(),axis=1)

In [17]:
df2.shape

(51336, 54)

In [18]:
for i in df2.columns:
    df2=df2.loc[df2[i]!=-99999]

In [19]:
df2.shape

(42066, 54)

In [20]:
df2.isna().sum().sum() ## all the null values resolved

0

In [21]:
# common columns in df1 snd df2 
for i  in df1.columns:
    if i in df2.columns:
        print(i)



PROSPECTID


In [22]:
#merge df1 snd df2 on the basis of common column 
df=pd.merge(df1,df2, how='inner', left_on=['PROSPECTID'],right_on=['PROSPECTID'])

In [23]:
df.sample(5)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
34812,42494,2,0,2,0,0,0.0,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,others,680,P2
8839,10708,1,0,1,0,0,0.0,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,others,others,672,P2
38875,47462,4,4,0,0,1,0.0,0.250,0.000,1.000,...,0.0,0.0,0.0,0.0,1,0,ConsumerLoan,ConsumerLoan,670,P2
36059,44024,1,1,0,0,1,0.0,1.000,0.000,1.000,...,0.0,0.0,0.0,0.0,0,0,others,others,668,P3
24165,29520,15,10,5,3,1,0.2,0.067,0.333,0.667,...,0.0,0.0,0.0,0.0,1,0,others,PL,681,P2


In [24]:
df.isna().sum().sum() #no nan values

0

In [25]:
# devide the features into categorical and numerical

cat=[]
num=[]
for i in df.columns:
    if df[i].dtype=='object':
        cat.append(i)
    else:
        num.append(i)

print("number of categorical colums:",len(cat),cat)
print("number of numerical colums:",len(num),num)


number of categorical colums: 6 ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2', 'Approved_Flag']
number of categorical colums: 73 ['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'num_times_delinquent', 'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts', 'num_deliq_6_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss', 'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq', 'CC_enq', 'CC_enq_

# See if categorical column have any effect on Approved Flag 
1. effect of 'MARITALSTATUS' on Approved_Flag using chi-Square test

** Alpha(assumed) significance level or strictness level usually 5% but in very critical projects alpha(marginal error) will be very very less.

** confidence intervel =1-alpha (how much you are confident about your inference)

** calculate evidence(p value) against nulll hypothesis, p-val(probability of null hypo to be true)

** p-value is calculated using T-test,ChiSquare,Anova

** compare p val with alpha, if p val< alpha the reject null hypo otherwise fails to reject the null hypothesis.
 

In [26]:
for i in cat:
    chi2, pval,_,_=chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i,'----', pval)

MARITALSTATUS ---- 3.5781808610388605e-233
EDUCATION ---- 2.6942265249737532e-30
GENDER ---- 1.9079361001865664e-05
last_prod_enq2 ---- 0.0
first_prod_enq2 ---- 7.849976105554191e-287
Approved_Flag ---- 0.0


All p-vals are less than 0.05, therefore all the categorical features have effect on 'Approved_Flag',we will keep all the categorical values.

# Checking the numerical features 
1. multicollinearity(VIF)=Predictablity of one feature by other feature 
2. here we will not use corelation between two columns beacause it will show only linear relationship bet considered columns, in case of convex functions correlation give misleading information. 
3. apply anova test to see the effect of num columns on 'Approved_Flag'

In [27]:
num_col=[]
for i in df.columns:
    if df[i].dtype!='object' and i not in ['PROSPECTID','Approved_Flag']:
        num_col.append(i)

len(num_col)

72

In [28]:
num_colum_to_be_kept = []
vif_data = df[num_col]
total_column=vif_data.shape[1]
col_index=0
#vif sequential check (it is better than parralel VIF)
for i in range(0,total_column):
    vif_val = variance_inflation_factor(vif_data, col_index)
    print(col_index, "------", vif_val)

    if vif_val < 6:
        num_colum_to_be_kept.append(num_col[i])
        col_index=col_index+1
    else:
        vif_data = vif_data.drop([num_col[i]], axis=1)

print("Columns to be kept:", num_colum_to_be_kept)
print('independent numerical columns:', len(num_colum_to_be_kept))


/Users/ravina/Desktop/banking credit risk modeling/creditvenv/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ------ inf


/Users/ravina/Desktop/banking credit risk modeling/creditvenv/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ------ inf
0 ------ 11.320180023968081
0 ------ 8.363698035000327
0 ------ 6.520647877790933
0 ------ 5.14950161821264
1 ------ 2.6111110405797304


/Users/ravina/Desktop/banking credit risk modeling/creditvenv/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 ------ inf
2 ------ 1788.7926256209232
2 ------ 8.601028256477221
2 ------ 3.8328007921530802
3 ------ 6.0996533816466565
3 ------ 5.581352009642807
4 ------ 1.9855843530987887


/Users/ravina/Desktop/banking credit risk modeling/creditvenv/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 ------ inf
5 ------ 4.809538302819327
6 ------ 23.270628983464515
6 ------ 30.595522588100156
6 ------ 4.384346405965592
7 ------ 3.06465841552343
8 ------ 2.8986397712992584
9 ------ 4.3778769153473345
10 ------ 2.207853583695844
11 ------ 4.916914200506874
12 ------ 5.21470203006474
13 ------ 3.3861625024231325
14 ------ 7.840583309478997
14 ------ 5.255034641721447


/Users/ravina/Desktop/banking credit risk modeling/creditvenv/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 ------ inf
15 ------ 7.380634506427226
15 ------ 1.4210050015175784
16 ------ 8.083255010190294
16 ------ 1.6241227524040025
17 ------ 7.257811920140027
17 ------ 15.596243832682953
17 ------ 1.825857047132433
18 ------ 1.5080839450032613
19 ------ 2.1720888348245726
20 ------ 2.623397553527229
21 ------ 2.2959970812106123
22 ------ 7.3605783191964695
22 ------ 2.1602387773102554
23 ------ 2.8686288267891613
24 ------ 6.45821800363729
24 ------ 2.8474118865638256
25 ------ 4.753198156284073
26 ------ 16.22735475594819
26 ------ 6.424377256363854
26 ------ 8.887080381808678
26 ------ 2.380474614295268
27 ------ 8.609513476514548
27 ------ 13.06755093547673
27 ------ 3.5000400566546555
28 ------ 1.9087955874813693
29 ------ 17.006562234161628
29 ------ 10.730485153719172
29 ------ 2.353849752295021
30 ------ 22.10485591513638
30 ------ 2.7971639638512977
31 ------ 3.424171203217695
32 ------ 10.17502145445105
32 ------ 6.408710354561287
32 ------ 1.0011511962625617
33 ------ 3.0691973

1. Original num val 72 
2. after vif 39

In [29]:
# Anova test for columns to be kept (one way anova)
from scipy.stats import f_oneway
colum_tobe_kept_num=[]
for i in num_colum_to_be_kept:
    a=list(df[i])
    b=list(df['Approved_Flag'])

    group_p1=[val for val,group in zip(a,b) if group=='P1']
    group_p2=[val for val,group in zip(a,b) if group=='P2']
    group_p3=[val for val,group in zip(a,b) if group=='P3']
    group_p4=[val for val,group in zip(a,b) if group=='P4']

    f_statistics,p_value=f_oneway(group_p1,group_p2,group_p3,group_p4)

    if p_value<0.05:
        colum_tobe_kept_num.append(i)

In [30]:
print(len(colum_tobe_kept_num))
# two more columnns reduced

37


In [31]:
features=colum_tobe_kept_num+['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df=df[features+['Approved_Flag']]
df.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0.000,0.0,0,0.000,0,0,0,4,1,4,...,0.000,0.0,1,0,Married,12TH,M,PL,PL,P2
1,0.000,0.0,0,0.000,0,0,0,0,0,1,...,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,P2
2,0.125,0.0,0,0.000,1,0,0,0,2,6,...,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others,P2
3,0.000,0.0,0,0.000,0,0,0,0,3,0,...,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL,P1
4,0.000,0.0,1,0.167,0,0,0,0,6,0,...,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL,P3


1. we filtered out(features selection) the numerical and categorical columns using Chi-square, VIF amd anova test 
2. we will see if there is any scope for further feature engineering

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [33]:
# verifying the categorical data 
for i in cat:
    print(i,":", df[i].unique())


MARITALSTATUS : ['Married' 'Single']
EDUCATION : ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
GENDER : ['M' 'F']
last_prod_enq2 : ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
first_prod_enq2 : ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']
Approved_Flag : ['P2' 'P1' 'P3' 'P4']


In [34]:
#Label encoding for EDUCATION
ed_dict={'12TH':2, 'GRADUATE':3, 'SSC':1, 'POST-GRADUATE':4, 'UNDER GRADUATE':3, 'OTHERS':1,
 'PROFESSIONAL':3}

df.loc[df['EDUCATION']=="SSC",['EDUCATION']]            =1
df.loc[df['EDUCATION']=='12TH',['EDUCATION']]           =2
df.loc[df['EDUCATION']=='GRADUATE',['EDUCATION']]       =3
df.loc[df['EDUCATION']=='UNDER GRADUATE',['EDUCATION']] =3
df.loc[df['EDUCATION']=='POST-GRADUATE',['EDUCATION']]  =4
df.loc[df['EDUCATION']=='PROFESSIONAL',['EDUCATION']]   =3
df.loc[df['EDUCATION']=='OTHERS',['EDUCATION']]         =1


df['EDUCATION'].value_counts()
df['EDUCATION'].astype(int)

0        2
1        3
2        1
3        4
4        2
        ..
42059    2
42060    1
42061    1
42062    3
42063    1
Name: EDUCATION, Length: 42064, dtype: int64

In [35]:
"""df["MARITALSTATUS"] = df["MARITALSTATUS"].map({True: "Married", False: "Single"})
df["GENDER"] = df["GENDER"].map({True: "Male", False: "Female"})
# Assuming "last_prod_enq2" and "first_prod_enq2" are boolean columns as well
df["last_prod_enq2"] = df["last_prod_enq2"].map({True: "LastProdTrue", False: "LastProdFalse"})
df["first_prod_enq2"] = df["first_prod_enq2"].map({True: "FirstProdTrue", False: "FirstProdFalse"})

# Now apply get_dummies"""

'df["MARITALSTATUS"] = df["MARITALSTATUS"].map({True: "Married", False: "Single"})\ndf["GENDER"] = df["GENDER"].map({True: "Male", False: "Female"})\n# Assuming "last_prod_enq2" and "first_prod_enq2" are boolean columns as well\ndf["last_prod_enq2"] = df["last_prod_enq2"].map({True: "LastProdTrue", False: "LastProdFalse"})\ndf["first_prod_enq2"] = df["first_prod_enq2"].map({True: "FirstProdTrue", False: "FirstProdFalse"})\n\n# Now apply get_dummies'

In [36]:
# one hot encode other categorical features 
df_encoded=pd.get_dummies(df,columns=["MARITALSTATUS","GENDER","last_prod_enq2","first_prod_enq2"],drop_first=True)

In [37]:
pd.options.display.max_columns = None

In [38]:
df_encoded.head(3)

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION,Approved_Flag,MARITALSTATUS_Single,GENDER_M,last_prod_enq2_CC,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.0,0,0.0,0,0,0,4,1,4,0,72,18,549,29,0,0,11,0,0,0,0,0,0,29,0,0,566,0,51000,114,0,1,0.0,0.0,1,0,2,P2,False,True,False,False,False,True,False,False,False,False,True,False
1,0.000,0.0,0,0.0,0,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,209,0,19000,50,0,0,0.0,0.0,0,0,3,P2,True,False,False,True,False,False,False,False,True,False,False,False
2,0.125,0.0,0,0.0,1,0,0,0,2,6,0,47,2,302,25,8,0,10,0,0,0,0,0,0,25,0,0,587,0,18,191,0,0,0.0,0.0,1,0,1,P2,False,True,False,True,False,False,False,False,False,False,False,True


In [39]:
df_encoded.shape

(42064, 51)

In [40]:
df_encoded['EDUCATION'] = pd.to_numeric(df_encoded['EDUCATION'], errors='coerce')

# Check for NaN values
print("NaN values in EDUCATION column:", df_encoded['EDUCATION'].isna().sum())

# Drop rows with NaN values (if necessary)
df_encoded = df_encoded.dropna(subset=['EDUCATION'])

# Now convert to integer type again
df_encoded['EDUCATION'] = df_encoded['EDUCATION'].astype(int)

NaN values in EDUCATION column: 0


In [41]:
df_encoded['EDUCATION'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 42064 entries, 0 to 42063
Series name: EDUCATION
Non-Null Count  Dtype
--------------  -----
42064 non-null  int64
dtypes: int64(1)
memory usage: 328.8 KB


# Model Building

In [42]:
y=df_encoded['Approved_Flag']
X=df_encoded.drop(['Approved_Flag'],axis=1)

In [43]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Decision Tree Classifier

In [44]:
y=df_encoded['Approved_Flag']
X=df_encoded.drop(['Approved_Flag'],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
DT_Cls=DecisionTreeClassifier(max_depth=20,min_samples_split=10,random_state=42)


In [45]:
DT_Cls.fit(X_train,y_train)
y_pred=DT_Cls.predict(X_test)


In [46]:
accuracy=accuracy_score(y_test,y_pred)
precision,recall,f1_score,_=precision_recall_fscore_support(y_test,y_pred)
print("Accuracy of the model is:", accuracy)
print()
for i,v in enumerate(['p1','p2','p3','p4']):
    print(f"Class {v}:")
    print(f"pricision:{precision[i]}")
    print(f"recall:{recall[i]}")
    print(f"F1 score:{f1_score[i]}")
    print()

Accuracy of the model is: 0.7096160703672887

Class p1:
pricision:0.72
recall:0.727810650887574
F1 score:0.7238842569887199

Class p2:
pricision:0.8073858114674441
recall:0.8233894945490585
F1 score:0.8153091265947007

Class p3:
pricision:0.3431528662420382
recall:0.32528301886792454
F1 score:0.33397907787679193

Class p4:
pricision:0.6555217831813577
recall:0.6287657920310982
F1 score:0.6418650793650794



# Random Forest

In [47]:
rf_classifier=RandomForestClassifier(n_estimators=200,random_state=42)

In [48]:
rf_classifier.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [49]:
y_pred=rf_classifier.predict(X_test)

In [50]:
accuracy=accuracy_score(y_test,y_pred)
precision,recall,f1_score,_=precision_recall_fscore_support(y_test,y_pred)


In [51]:
print("Accuracy of the model is:", accuracy)
print()
for i,v in enumerate(['p1','p2','p3','p4']):
    print(f"Class {v}:")
    print(f"pricision:{precision[i]}")
    print(f"recall:{recall[i]}")
    print(f"F1 score:{f1_score[i]}")
    print()

Accuracy of the model is: 0.7663140377986449

Class p1:
pricision:0.8411214953271028
recall:0.7100591715976331
F1 score:0.7700534759358288

Class p2:
pricision:0.7974855589534489
recall:0.930426164519326
F1 score:0.8588418259994511

Class p3:
pricision:0.4390625
recall:0.2120754716981132
F1 score:0.2860050890585242

Class p4:
pricision:0.7293889427740058
recall:0.7308066083576288
F1 score:0.7300970873786408



# Xgboost

In [52]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

In [53]:
df_encoded['EDUCATION'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 42064 entries, 0 to 42063
Series name: EDUCATION
Non-Null Count  Dtype
--------------  -----
42064 non-null  int64
dtypes: int64(1)
memory usage: 328.8 KB


In [54]:
#y=df_encoded['Approved_Flag']
y_encoded=label_encoder.fit_transform(df_encoded['Approved_Flag'])
X=df_encoded.drop(['Approved_Flag'],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2,random_state=42)

In [55]:

xgb_classifier = xgb.XGBClassifier()

# Training the model
xgb_classifier.fit(X_train, y_train)

# Making predictions
y_pred = xgb_classifier.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7752288125520028


In [56]:
accuracy=accuracy_score(y_test,y_pred)
precision,recall,f1_score,_=precision_recall_fscore_support(y_test,y_pred)

print("Accuracy of the model is:", accuracy)
print()
for i,v in enumerate(['p1','p2','p3','p4']):
    print(f"Class {v}:")
    print(f"pricision:{precision[i]}")
    print(f"recall:{recall[i]}")
    print(f"F1 score:{f1_score[i]}")
    print()

Accuracy of the model is: 0.7752288125520028

Class p1:
pricision:0.825346112886049
recall:0.7642998027613412
F1 score:0.7936507936507936

Class p2:
pricision:0.8238249013275923
recall:0.9102081268582756
F1 score:0.8648648648648649

Class p3:
pricision:0.46108490566037735
recall:0.29509433962264153
F1 score:0.3598711458812701

Class p4:
pricision:0.7262357414448669
recall:0.7424684159378037
F1 score:0.7342623738587217



# Accuracy Values:
1. DT: approx 71
2. RandomForest: 76
3. Xgboost: approx 78

** Xgboost is giving the best accuracy for now, so we will further fine tune it. we will do hyper parameter tuning.


In [57]:
df_encoded['Approved_Flag'].value_counts()
# target valrriable 'Approved_Flag' is relatively balanced

Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64

# Hyperparameter: It decides how fast the algorithm want to converge
l.r=reduce the overfitting effect  

In [62]:
params_grid={
    'colsample_bytree': [0.1,0.3,0.5,0.7,0.9],
    'learning_rate': [0.001,0.01,0.1,1],
    'max_depth': [3,5,8,10],
    'alpha': [1,10,100],
    'n_estimators': [10,50,100]
}

index=0

ans_grid={
    'combination':[],
    'train_accuracy':[],
    'test_accuracy':[],
    'colsample_bytree':[],
    'learning_rate':[],
    'max_depth':[],
    'n_estimators':[],
    'alpha':[]
}

In [65]:
# Looping through each combination of the hyperparameters 
for colsample_bytree in params_grid['colsample_bytree']:
    for learning_rate in params_grid['learning_rate']:
       for max_depth in params_grid['max_depth']:
           for alpha in params_grid['alpha']:
               for n_estimators in params_grid['n_estimators']:

                   index=index+1 

                   #let's define and train the Xgboost model 
                   model=xgb.XGBClassifier(objective='multi:softmax',
                                           num_class=4,
                                           colsample_bytree=colsample_bytree,
                                           learning_rate=learning_rate,
                                           max_depth=max_depth,
                                           alpha=alpha,
                                           n_estimators=n_estimators)
                   
                   y=df_encoded['Approved_Flag']
                   X=df_encoded.drop(['Approved_Flag'],axis=1)

                   label_encoder=LabelEncoder()
                   y_encoded=label_encoder.fit_transform(y)


                   X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2,random_state=42)

                   model.fit(X_train,y_train)
                   #predicting on trai ing and test data 
                   y_pred_train=model.predict(X_train)
                   y_pred_test=model.predict(X_test)

                   # accuracy on train and test results 
                   train_accuracy=accuracy_score(y_train,y_pred_train)
                   test_accuracy=accuracy_score(y_test,y_pred_test)

                   #updating into the list

                   ans_grid['combination'].append(index)
                   ans_grid['train_accuracy'].append(train_accuracy)
                   ans_grid['test_accuracy'].append(test_accuracy)
                   ans_grid['colsample_bytree'].append(colsample_bytree)
                   ans_grid['learning_rate'].append(learning_rate)
                   ans_grid['max_depth'].append(max_depth)
                   ans_grid['alpha'].append(alpha)
                   ans_grid['n_estimators'].append(n_estimators)



                   #results for the combinations 

                   



In [75]:
import pandas as pd

# Initialize an empty DataFrame
params_Out = pd.DataFrame(ans_grid)
# Append rows to the DataFrame
params_Out.tail()

,combination,train_accuracy,test_accuracy,colsample_bytree,learning_rate,max_depth,n_estimators,alpha
1613,1615,0.937595,0.765244,0.9,1.0,10,50,10
1614,1616,0.962230,0.768097,0.9,1.0,10,100,10
1615,1617,0.783246,0.770237,0.9,1.0,10,10,100
1616,1618,0.787406,0.772970,0.9,1.0,10,50,100
1617,1619,0.787406,0.772970,0.9,1.0,10,100,100


In [76]:
params_Out.shape

(1618, 8)

In [71]:
# Find the index of the row with the maximum train accuracy
best_train_index = params_Out['train_accuracy'].idxmax()

# Find the index of the row with the maximum test accuracy
best_test_index = params_Out['test_accuracy'].idxmax()

# Get the rows with the best train and test accuracy
best_train_row = params_Out.loc[best_train_index]
best_test_row = params_Out.loc[best_test_index]

print("Row with the best train accuracy:")
print(best_train_row)

print("\nRow with the best test accuracy:")
print(best_test_row)


Row with the best train accuracy:
combination         280.000000
train_accuracy        0.999970
test_accuracy         0.751813
colsample_bytree      0.300000
learning_rate         1.000000
max_depth            10.000000
n_estimators         10.000000
alpha                 1.000000
Name: 279, dtype: float64

Row with the best test accuracy:
combination         1588.000000
train_accuracy         0.802175
test_accuracy          0.780815
colsample_bytree       0.900000
learning_rate          1.000000
max_depth              3.000000
n_estimators          50.000000
alpha                 10.000000
Name: 1586, dtype: float64


In [78]:
# Let's take the above combination for the highest test accuraccy 
model=xgb.XGBClassifier(objective='multi:softmax',
                        num_class=4,
                        colsample_bytree=0.9,
                        learning_rate=1,
                        max_depth=3,
                        alpha=10,
                        n_estimattors=50)

In [84]:
unseen_data="https://github.com/ravina029/datasets/raw/main/Unseen_Dataset.xlsx"


In [86]:

df_unseen=pd.read_excel(unseen_data)

In [87]:
df_unseen.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2
0,0.000,0.0,0,0.000,0,0,0,4,1,4,0,72,18,549,29,0,0,11,0,0,0,0,0,0,29,0,0,566,0,51000,114,0,1,0.000,0.0,1,0,Married,12TH,M,PL,PL
1,0.000,0.0,0,0.000,0,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,209,0,19000,50,0,0,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan
2,0.125,0.0,0,0.000,1,0,0,0,2,6,0,47,2,302,25,8,0,10,0,0,0,0,0,0,25,0,0,587,0,18,191,0,0,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others
3,0.000,0.0,0,0.000,0,0,0,0,3,0,2,131,32,583,0,0,0,16,0,0,0,0,0,0,0,0,0,3951,0,15000,75,0,0,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL
4,0.000,0.0,1,0.167,0,0,0,0,6,0,0,150,17,245,270,0,11,2,3,0,1,0,0,0,26,1,3,7,4,0,154,0,0,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL


In [89]:
df_unseen.isnull().sum().sum()

0

In [90]:
df_unseen.iloc[42]

pct_tl_open_L6M                 0.0
pct_tl_closed_L6M               0.0
Tot_TL_closed_L12M                0
pct_tl_closed_L12M              0.0
Tot_Missed_Pmnt                   0
CC_TL                             0
Home_TL                           0
PL_TL                             0
Secured_TL                        1
Unsecured_TL                      0
Other_TL                          0
Age_Oldest_TL                    44
Age_Newest_TL                    44
time_since_recent_payment       518
max_recent_level_of_deliq        23
num_deliq_6_12mts                 0
num_times_60p_dpd                 0
num_std_12mts                     0
num_sub                           0
num_sub_6mts                      0
num_sub_12mts                     0
num_dbt                           0
num_dbt_12mts                     0
num_lss                           0
recent_level_of_deliq            23
CC_enq_L12m                       0
PL_enq_L12m                       0
time_since_recent_enq       

In [98]:
col_unseen=list(df_unseen.columns)


'first_prod_enq2'

In [91]:
col_unseen=list(df_unseen.columns)

print("MARITALSTATUS",df_unseen['MARITALSTATUS'].unique())
df_unseen['MARITALSTATUS'].unique()
df_unseen['GENDER'].unique()
df_unseen['last_prod_enq2'].unique()
df_unseen['first_prod_enq2'].unique()

array(['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC'], dtype=object)

In [99]:
df_unseen.loc[df_unseen['EDUCATION']=="SSC",['EDUCATION']]            =1
df_unseen.loc[df_unseen['EDUCATION']=='12TH',['EDUCATION']]           =2
df_unseen.loc[df_unseen['EDUCATION']=='GRADUATE',['EDUCATION']]       =3
df_unseen.loc[df_unseen['EDUCATION']=='UNDER GRADUATE',['EDUCATION']] =3
df_unseen.loc[df_unseen['EDUCATION']=='POST-GRADUATE',['EDUCATION']]  =4
df_unseen.loc[df_unseen['EDUCATION']=='PROFESSIONAL',['EDUCATION']]   =3
df_unseen.loc[df_unseen['EDUCATION']=='OTHERS',['EDUCATION']]         =1


df_unseen['EDUCATION'].value_counts()
df_unseen['EDUCATION']=df_unseen['EDUCATION'].astype(int)
df_unseen['EDUCATION'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: EDUCATION
Non-Null Count  Dtype
--------------  -----
100 non-null    int64
dtypes: int64(1)
memory usage: 928.0 bytes


In [100]:
df_unseen_encoded=pd.get_dummies(df_unseen,columns=["MARITALSTATUS","GENDER","last_prod_enq2","first_prod_enq2"],drop_first=True)

In [102]:
df_unseen_encoded.describe()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION
count,100.000000,100.000000,100.00000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.0,100.00,100.00,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.191230,0.116210,0.71000,0.16822,0.410000,0.090000,0.040000,0.410000,2.330000,2.300000,1.140000,42.980000,13.000000,181.750000,22.800000,0.23000,0.410000,3.630000,0.080000,0.0,0.01,0.11,0.0,0.0,19.670000,0.170000,0.790000,287.920000,1.09000,25764.300000,108.330000,0.08000,0.230000,0.154610,0.093330,0.160000,0.030000,2.250000
std,0.294655,0.253891,1.13079,0.27274,0.792579,0.320826,0.242878,1.005992,4.811718,2.544553,2.470104,36.271139,14.595558,270.985943,63.869659,0.90849,1.551116,7.473394,0.580491,0.0,0.10,1.10,0.0,0.0,58.377569,0.493493,1.793901,503.740607,1.83179,13707.266015,65.996029,0.27266,0.422953,0.335204,0.279607,0.368453,0.171447,0.903137
min,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000,16.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,13.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,4.000000,54.750000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000,0.000000,20.500000,0.00000,15875.000000,60.750000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,1.500000,0.000000,39.000000,8.000000,75.500000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000,0.000000,148.500000,0.00000,24000.000000,101.500000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,0.343500,0.091000,1.00000,0.25000,0.250000,0.000000,0.000000,0.000000,2.000000,3.000000,1.000000,59.000000,14.500000,143.000000,15.250000,0.00000,0.000000,5.000000,0.000000,0.0,0.00,0.00,0.0,0.0,15.250000,0.000000,1.000000,298.500000,1.00000,30000.000000,130.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.000000,1.000000,5.00000,1.00000,3.000000,2.000000,2.000000,7.000000,35.000000,10.000000,16.000000,191.000000,76.000000,1492.000000,478.000000,8.00000,11.000000,43.000000,5.000000,0.0,1.00,11.00,0.0,0.0,478.000000,3.000000,13.000000,3951.000000,10.00000,65000.000000,306.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


In [103]:
# Let's take the above combination for the highest test accuraccy 
model=xgb.XGBClassifier(objective='multi:softmax',
                        num_class=4,
                        colsample_bytree=0.9,
                        learning_rate=1,
                        max_depth=3,
                        alpha=10,
                        n_estimattors=50)

In [104]:
model.fit(X_train,y_train)

/Users/ravina/Desktop/banking credit risk modeling/creditvenv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:29:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "n_estimattors" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_estimattors=50,
              n_jobs=None, ...)

In [106]:
y_pred_unseen=model.predict(df_unseen_encoded)

In [108]:
type(y_pred)

numpy.ndarray

In [110]:
y_out = pd.DataFrame(y_pred, columns=['Pred_target'])

In [111]:
y_out.value_counts()

Pred_target
1              5574
3              1052
0               939
2               848
Name: count, dtype: int64

In [113]:
pd.DataFrame(y_encoded,columns=['orig_target']).value_counts()

orig_target
1              25452
2               6440
3               5264
0               4908
Name: count, dtype: int64

distribution of the predicted output on the unseen data is similar to the distribution of the original dataset  